# Scaling out: `xarray` & `dask`


<img src="https://github.com/pydata/xarray/raw/master/doc/_static/dataset-diagram.png" alt="xarray Logo" style="height: 150px;">
<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg" 
     width="30%" 
     align=right
     alt="Dask logo">

## Parallel computing with Dask
Dask divides arrays into many small pieces, called chunks, each of which is presumed to be small enough to fit into memory.

![Dask Arrays](http://dask.pydata.org/en/latest/_images/dask-array-black-text.svg)

_source: [Dask Array Documentation](http://dask.pydata.org/en/latest/array-overview.html)_

## Spinning up a cluster

```python
from dask_jobqueue import PBSCluster
PBSCluster(cores=1, memory='10GB', processes=1, queue='share', walltime='01:00:00')
```
cores: Total number of cores per job
memory: '109GB'             # Total amount of memory per job
processes: 9                # Number of Python processes per job
interface: ib0              # Network interface to use like eth0 or ib0
queue: regular
walltime: '04:00:00'